In [6]:
import numpy as np
import pandas as pd

In [33]:
dataset = 'astafev_2024'
infile = "../resources/"+dataset+"/SRR_per_SampleName.txt"
with open(infile, 'r') as f:
    sample_name = [l.split('\t')[0] for l in f.read().splitlines()[1:]]

intables = [f'../results/{dataset}/kallisto/{s}/abundance.tsv' for s in sample_name]

In [104]:
# concatenate all tables
abundance = pd.DataFrame()
for table in intables:
    tmp = pd.read_csv(table, sep='\t', index_col=0, usecols=['target_id', 'tpm'])
    tmp.columns = [table.split('/')[-2]]
    abundance = pd.concat([abundance, tmp], axis=1)

del tmp

# split between mRNA and pre-mRNA
premrna_idx = [id[:11]=="pre_ENSMUST" for id in abundance.index]
mrna_idx = [id[:7]=="ENSMUST" for id in abundance.index]

assert sum(premrna_idx) + sum(mrna_idx) == len(abundance.index)
premrna = abundance.loc[premrna_idx,:]
mrna = abundance.loc[mrna_idx,:]

In [61]:
promid_transcripts = pd.read_csv(f'../results/promoterome/mm10/promid_transcripts.txt', sep='\t', header=None)
promid_transcripts.columns = ['prom','transcript']

def remove_version(x):
    return x.split('.')[0]
promid_transcripts.loc[:,'transcript'] = promid_transcripts.loc[:,['transcript']].map(remove_version)

# fill in nan promoters
for i in promid_transcripts.index:
    if pd.isnull(promid_transcripts.iloc[i, 0]):
        promid_transcripts.iloc[i, 0] = promid_transcripts.iloc[i-1, 0]

# verify that all transcripts are unique
assert len(promid_transcripts.transcript.unique()) == len(promid_transcripts)

# make dictionary
#my_dict = promid_transcripts.set_index('transcript').to_dict()
my_dict = dict(zip(promid_transcripts.transcript.values, promid_transcripts.prom.values))

In [63]:
# get transcript ids as index
premrna.index = [id.split('::')[0].split('_')[1] for id in premrna.index]
premrna.index = premrna.index.map(remove_version)

mrna.index = [id.split('|')[0] for id in mrna.index]
mrna.index = mrna.index.map(remove_version)


ValueError: Lengths must match to compare

In [ ]:
    # Read in kallisto tpm tables and get transcript ids as index
    tables = {}
    tables['premrna'] = pd.read_csv(args.premrna, sep='\t', index_col=0)
    tables['premrna'].index = [id.split('::')[0].split('_')[1] for id in tables['premrna'].index]
    tables['premrna'].index = tables['premrna'].index.map(remove_version)

    tables['mrna'] = pd.read_csv(args.mrna, sep='\t', index_col=0)
    tables['mrna'].index = [id.split('|')[0] for id in tables['mrna'].index]
    tables['mrna'].index = tables['mrna'].index.map(remove_version)

In [73]:
idx_in = np.array([id in my_dict for id in premrna.index])

In [75]:
np.sum(idx_in)/premrna.shape[0]

np.float64(0.7946486237121329)

In [92]:
idx_premrna_only = list(set(premrna.index) - set(mrna.index))

In [93]:
premrna.loc[idx_in,:]

,F_ZT2_r1,F_ZT2_r2,F_ZT2_r3,F_ZT6_r1,F_ZT6_r2,F_ZT6_r3,F_ZT10_r1,F_ZT10_r2,F_ZT10_r3,F_ZT14_r1,...,M_ZT10_r3,M_ZT14_r1,M_ZT14_r2,M_ZT14_r3,M_ZT18_r1,M_ZT18_r2,M_ZT18_r3,M_ZT22_r1,M_ZT22_r2,M_ZT22_r3
ENSMUST00000070533,0.001550,0.003608,0.001797,0.006000,0.001506,0.002960,0.001416,0.002581,0.001079,0.002794,...,0.004539,0.002629,0.003779,0.001401,0.002043,0.00115,0.001518,0.002756,0.000807,0.002590
ENSMUST00000027032,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
ENSMUST00000027035,0.241584,0.589433,0.267510,0.471351,0.161013,0.168450,0.000000,0.163831,0.098797,0.344610,...,0.222105,0.054925,0.658367,0.180131,0.656983,0.36671,0.353921,0.000000,0.000000,0.384411
ENSMUST00000116652,0.000000,0.000000,0.000000,0.000000,0.023748,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.283355,0.164382,0.000000,0.408372,0.000000,1.11264,0.131112,0.570799,0.037802,0.058658
ENSMUST00000130201,0.365234,0.420819,0.200631,0.528294,0.189500,0.195129,0.000373,0.362417,0.227834,0.473728,...,0.311460,0.245919,0.337246,0.313658,0.273648,0.00000,0.277644,0.210574,0.125483,0.097714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSMUST00000181205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
ENSMUST00000181163,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
ENSMUST00000177666,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
ENSMUST00000178181,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000


In [98]:
my_dict.keys()

dict_keys(['ENSMUST00000027036', 'ENSMUST00000119612', 'ENSMUST00000137887', 'ENSMUST00000115529', 'ENSMUST00000150971', 'ENSMUST00000131119', 'ENSMUST00000081551', 'ENSMUST00000044369', 'ENSMUST00000160777', 'ENSMUST00000027038', 'ENSMUST00000160339', 'ENSMUST00000027040', 'ENSMUST00000133144', 'ENSMUST00000140079', 'ENSMUST00000131494', 'ENSMUST00000043578', 'ENSMUST00000131467', 'ENSMUST00000150761', 'ENSMUST00000151281', 'ENSMUST00000155921', 'ENSMUST00000139838', 'ENSMUST00000061280', 'ENSMUST00000182114', 'ENSMUST00000072079', 'ENSMUST00000144177', 'ENSMUST00000088666', 'ENSMUST00000168907', 'ENSMUST00000166384', 'ENSMUST00000171265', 'ENSMUST00000097826', 'ENSMUST00000171802', 'ENSMUST00000052843', 'ENSMUST00000125294', 'ENSMUST00000140948', 'ENSMUST00000122156', 'ENSMUST00000118263', 'ENSMUST00000119714', 'ENSMUST00000149214', 'ENSMUST00000027056', 'ENSMUST00000048613', 'ENSMUST00000171690', 'ENSMUST00000137824', 'ENSMUST00000135014', 'ENSMUST00000179089', 'ENSMUST00000180062',

In [103]:
sum([id in my_dict for id in mrna.index])/mrna.shape[0]

0.7955784093358377

In [102]:
sum([id in mrna.index for id in my_dict.keys()])/len(my_dict)

0.6276386426020842